In [6]:
from ultralytics import YOLO
import cv2
import winsound     # alarm (Windows only)
import csv
import time

class HumanDetectionBackend:
    def __init__(self, model_path="model.pt", video_path="C:/Users/dell/Desktop/Uraan/HUMAN_DETECTION/Test/test.mp4", log_file="detections_log.csv"):
        self.model_path = model_path
        self.video_path = video_path
        self.log_file = log_file

        # frame aur timing
        self.frame_count = 0
        self.start_time = time.time()

        # run full pipeline
        self.load_model()
        self.open_video()
        self.prepare_log()
        self.run_detection()

    def load_model(self):
        print("[INFO] Loading YOLO model...")
        self.model = YOLO(self.model_path)

    def open_video(self):
        print(f"[INFO] Opening video: {self.video_path}")
        self.cap = cv2.VideoCapture(self.video_path)
        if not self.cap.isOpened():
            raise Exception("Error: Cannot open video file!")

    def prepare_log(self):
        print(f"[INFO] Preparing log file: {self.log_file}")
        self.log_file_obj = open(self.log_file, mode="w", newline="")
        self.csv_writer = csv.writer(self.log_file_obj)
        self.csv_writer.writerow(["frame_number", "timestamp_sec", "label", "confidence"])

    def play_alarm(self):
        duration = 500  # ms
        freq = 1000     # Hz
        winsound.Beep(freq, duration)

    def run_detection(self):
        print("[INFO] Starting detection...")
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("[INFO] Video ended.")
                break

            self.frame_count += 1
            timestamp = time.time() - self.start_time

            # Run YOLO inference
            results = self.model(frame, conf=0.6, verbose=False)
            annotated = results[0].plot()

            # Check detections
            for box in results[0].boxes:
                cls_id = int(box.cls[0].item())
                conf = float(box.conf[0].item())
                label = self.model.names[cls_id]

                # Log in CSV
                self.csv_writer.writerow([self.frame_count, round(timestamp, 2), label, round(conf, 2)])

                # Trigger alarm if "human"
                if label.lower() in ["person", "human"]:
                    self.play_alarm()

            # Show video
            cv2.imshow("YOLOv8 Human Detection", annotated)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                print("[INFO] Stopped by user.")
                break

        # cleanup
        self.cleanup()

    def cleanup(self):
        print("[INFO] Releasing resources...")
        self.cap.release()
        self.log_file_obj.close()
        cv2.destroyAllWindows()


if __name__ == "__main__":
     HumanDetectionBackend()


[INFO] Loading YOLO model...
[INFO] Opening video: C:/Users/dell/Desktop/Uraan/HUMAN_DETECTION/Test/test.mp4
[INFO] Preparing log file: detections_log.csv
[INFO] Starting detection...


KeyboardInterrupt: 